In [2]:
import numpy as np
import pandas as pd
import re
from numpy.random import seed
import os
from scipy import stats
from numpy.random import randint
import shutil

# Read Questionnaires data

In [7]:
# We updated the questionniare on January 20, 2024. 
# So we saved the answers until the 20 and combined them to the new responders on the new version
# For all old responders we added a None values on all the questions we add in the new version
exp_data = pd.read_csv('questionnaire_raw_data/questionnaire_before_reserve_ques.csv')
exp_data = exp_data.drop(exp_data[exp_data['DistributionChannel']=='preview'].index) # remove previews
exp_data = exp_data.dropna(subset=['uniqeID']) # remove rows without uniqeID

# this data is before the reserve questions added to the war questionnaire
# exp_data_old_version = pd.read_csv('questionnaire_raw_data/questionnaire_May_17.csv')
exp_data_old_version = pd.read_csv('questionnaire_raw_data/questionnaire_June_19.csv')
exp_data_old_version = exp_data_old_version.drop(exp_data_old_version[exp_data_old_version['DistributionChannel']=='preview'].index) # remove previews
exp_data_old_version = exp_data_old_version.dropna(subset=['uniqeID']) # remove rows without uniqeID

data_df = exp_data.combine_first(exp_data_old_version)

# fixing an error in the questionnaire in 1 question instead במידה מועטה רבה
# to be במידה מועטה
data_df.replace('במידה מועטה רבה', 'במידה מועטה', inplace=True)

# participant number 132 confused and write 123 instead 132
two_participants_id = data_df.loc[data_df['uniqeID']=='123']
# fixing the confusion
data_df['uniqeID'][data_df['EndDate']=='2024-01-29 19:18:49'] = '132'


# participant 184 wrote with another ' ' so fixing it
data_df.replace(' 184', '184',inplace=True)

data_df.drop(67, inplace=True) # remove one questionnaire that was stucj to the participant and he start it over
data_df[data_df['uniqeID']=='140'] # chacking that I see only one row with uniqeID of 140

# participant 552 wrote in his uniqueID 551 instead
two_participants_id = data_df.loc[data_df['uniqeID']=='551'] # finding the participants with same id
data_df['uniqeID'][data_df['EndDate']=='2024-05-27 15:58:26'] = '552' # fixing the confusion

# rename the column "uniqeID" bc a tipo
data = data_df.rename(columns={'uniqeID': 'uniqueID'})

# Display the updated new version dataframe
data.head()


,DistributionChannel,Duration (in seconds),EndDate,Finished,Progress,RecordedDate,ResponseId,StartDate,Status,UserLanguage,...,war_jobStatusBefore,war_newsConsumption,war_newsResources,war_servForce,war_servForce2,war_servForce3,war_servForce3other,war_worry2someone,war_worry2someone2,war_worry2someone3
0,Distribution Channel,Duration (in seconds),End Date,Finished,Progress,Recorded Date,Response ID,Start Date,Response Type,User Language,...,"מצב תעסוקתי (בשגרה, לפני ה- 7/10/23)",כמה זמן את/ה מבלה מול החדשות ביום,מה מקור/ות החדשות העיקרי/ם שלך (ניתן לסמן כמה ...,האם שרת/ת בשירות צבאי/מילואים/כוחות הביטחון מא...,אורך השירות במלחמה (יש לציין מספר שבועות או מס...,תפקידי בשירות הינו:,"במידה וסימנת אחר, נשמח שתפרט/י במידת האפשר",האם יש א/נשים ספציפיים במעגל קרוב שאת/ה דואג/ת...,עד כמה הדאגה לא/נשים ספציפיים מעסיקה אותה במהל...,עד כמה הדאגה לא/נשים ספציפיים מפריעה לך לישון ...
1,"{""ImportId"":""distributionChannel""}","{""ImportId"":""duration""}","{""ImportId"":""endDate"",""timeZone"":""Europe/Athens""}","{""ImportId"":""finished""}","{""ImportId"":""progress""}","{""ImportId"":""recordedDate"",""timeZone"":""Europe/...","{""ImportId"":""_recordId""}","{""ImportId"":""startDate"",""timeZone"":""Europe/Ath...","{""ImportId"":""status""}","{""ImportId"":""userLanguage""}",...,"{""ImportId"":""QID1215542841""}","{""ImportId"":""QID1215542845""}","{""ImportId"":""QID1215542846""}","{""ImportId"":""QID1215660416""}","{""ImportId"":""QID1215660417_TEXT""}","{""ImportId"":""QID1215660418""}","{""ImportId"":""QID1215660419_TEXT""}","{""ImportId"":""QID1215542851""}","{""ImportId"":""QID1215542852""}","{""ImportId"":""QID1215542853""}"
19,anonymous,619,2023-12-17 17:17:18,True,100,2023-12-17 17:17:19,R_1R6hR2rt6cWkp6F,2023-12-17 17:06:58,IP Address,HE,...,משרה חלקית,עד 2 שעות,"רדיו,אתרי חדשות,מדיה חברתית",NaN,NaN,NaN,NaN,כן,במידה רבה,במידה בינונית
20,anonymous,947,2024-01-03 16:20:02,True,100,2024-01-03 16:20:03,R_2RdYEoBoKYX29sa,2024-01-03 16:04:15,IP Address,HE,...,משרה חלקית,עד 2 שעות,"רדיו,טלוויזיה",NaN,NaN,NaN,NaN,כן,במידה רבה,במידה בינונית
21,anonymous,610,2024-01-07 12:00:50,True,100,2024-01-07 12:00:51,R_2r8MhgLCoEhClwL,2024-01-07 11:50:39,IP Address,HE,...,משרה חלקית,עד 2 שעות,"אתרי חדשות,מדיה חברתית",NaN,NaN,NaN,NaN,כן,במידה בינונית,כלל לא


### Global Variables 

In [10]:
DF_ONLINE = pd.DataFrame()
DF_reSMART = pd.DataFrame()
DF_reBIOS = pd.DataFrame()
DF_newBIOS = pd.DataFrame()
DF_ONLINE_BIOS = pd.DataFrame()

### Divide the participants according to experiment

In [13]:
## need to filter out the relevant participant for each experiment:
# Online (1__) # Smarting Return () # Smarting New () # BioSemi Return () # BioSemi New ()

def assign_df_to_experiment(data):
    id_signs = data['uniqueID'].str[0].unique()
    
    df_online = pd.DataFrame()
    df_reSmart = pd.DataFrame()
    df_newSmart = pd.DataFrame()
    df_reBioS = pd.DataFrame()
    df_newBioS = pd.DataFrame()
    df_online_bios = pd.DataFrame()
    
    for id_sign in id_signs:
        if id_sign == '1':
            df_online = data.loc[data['uniqueID'].str.startswith('1'),:]
#             df_online_bios = data.loc[data['uniqeID'].str.startswith('1'),:]
            
        elif id_sign == 'R':
            df_reBioS = data.loc[data['uniqueID'].str.startswith('R'),:]
            
        elif id_sign == 'W':
            df_reSmart = data.loc[data['uniqueID'].str.startswith('W'),:]
            
        elif id_sign == '5':
            df_newBioS = data.loc[data['uniqueID'].str.startswith('5'),:]
#             df_online_bios = data.loc[data['uniqeID'].str.startswith('5'),:]
    df_online_bios = pd.concat([df_online, df_newBioS])
            
    return df_online, df_reSmart, df_reBioS, df_newBioS, df_online_bios

### Global variables

In [16]:
DF_ONLINE, DF_reSMART, DF_reBIOS, DF_newBIOS, DF_ONLINE_BIOS = assign_df_to_experiment(data)


In [18]:
# separate each DF to 6 dfs according to questionnaires

def separate_to_6_dfs (df):
    
    df_demo = pd.DataFrame()    
    df_war = pd.DataFrame()
    df_dass = pd.DataFrame()
    df_pcl = pd.DataFrame()
    df_cope = pd.DataFrame()
    df_asrs = pd.DataFrame()
    
    selected_demo_cols = df.filter(regex='^demo', axis=1)
    selected_war_cols = df.filter(regex='^war', axis=1)
    selected_dass_cols = df.filter(regex='^dass', axis=1)
    selected_pcl_cols = df.filter(regex='^pcl', axis=1)
    selected_cope_cols = df.filter(regex='^cope', axis=1)
    selected_asrs_cols = df.filter(regex='^asrs', axis=1)

    df_demo = df[selected_demo_cols.columns]
    df_war = df[selected_war_cols.columns]
    df_dass = df[selected_dass_cols.columns]
    df_pcl = df[selected_pcl_cols.columns]
    df_cope = df[selected_cope_cols.columns]
    df_asrs = df[selected_asrs_cols.columns]
    
    return df_demo, df_war, df_dass, df_pcl, df_cope, df_asrs

# Function to add the unique ID column to a DataFrame
def add_uniqueID_column(dfs, main_df):
    for df in dfs:
        df.insert(0, 'uniqueID', main_df['uniqueID'])
    
 


## Create separate DF for each questionnaire

### Global variables

In [22]:
all_demo,all_war,all_dass,all_pcl,all_cope,all_asrs = separate_to_6_dfs(data)
online_demo,online_war,online_dass,online_pcl,online_cope,online_asrs = separate_to_6_dfs(DF_ONLINE)
onbio_demo,onbio_war,onbio_dass,onbio_pcl,onbio_cope,onbio_asrs = separate_to_6_dfs(DF_ONLINE_BIOS)
add_uniqueID_column([onbio_demo,onbio_war,onbio_dass,onbio_pcl,onbio_cope,onbio_asrs], data)
# list(onbio_dass['uniqueID'])


#### Translation Dictionary 

In [25]:

demo_dict = {'זכר': 'male', 
             'נקבה': 'female', 
             'כן': 'yes', 
             'לא': 'no', 
             'אחר': 'other', 
             'במהלך תואר': 'during academy',
             'אקדמאית - בעל/ת תואר': 'academic',
             'תיכונית': 'high scholl'}

prior_diagnosis_categories = ['ADHD', 'PTSD', 'DASS', 'NO', 'OTHER'] # other include pain-killer and cancer drugs

war_dict = {'מידי יום': 'Every day',
            'בין פעמיים לשלוש בשבוע': 'Between 2-3 times a week',
            'מעט': 'Few',
            'בכלל לא': 'Not at all',
            'כן': 'Yes',
            'לא, אני מפונה ומתגורר/ת במלון/אכסנייה': 'No, I live in hotel',
            'לא, אני מפונה ומתגורר/ת בבית אחר': 'No, I live at a house that not main',
            'לא': 'No',
            'באופן חלקי': 'Partially',
            'לא צורך כלל': 'Not at all',
            'עד 2 שעות': 'Two hours max',
            'בין 2 ל- 3 שעות': 'Between 2 to 3 hours',
            '5 שעות ויותר': 'At least 5 hours',
            'כן, ואני עדיין בשירות פעיל': 'Yes, I still serve',
            'כן, וסיימתי את שירותי': 'Yes, I finished my reserve',
            'במידה רבה': 'A lot',
            'במידה רבה מאוד': 'Extremely',
            'במידה מועטה': 'Slightly',
            'במידה בינונית': 'Moderately'}

dass_dict = {'מתארת מאוד': 3,
             'מתארת באופן ניכר': 2,
             'מתארת באופן חלקי': 1,
             'לא מתארת כלל': 0}

pcl_dict = {'בכלל לא': 0,
            'במידה מועטה': 1,
            'במידה בינונית': 2,
            'במידה רבה': 3, 
            'במידה רבה מאוד': 4}

cope_dict = {'בכלל לא': 1,
             'במידה מועטה': 2,
             'במידה בינונית': 3,
             'במידה רבה': 4}

asrs_dict = {'אף פעם לא': 1,
             'לעיתים רחוקות': 2,
             'לפעמים': 3,
             'לעיתים תכופות': 4,
             'לעיתים תכופות מאוד': 5}


# Calculate the scores 

## Demo & War

In [29]:
onbio_prior_diagnosis = pd.DataFrame()
onbio_prior_diagnosis['uniqueID'] = onbio_demo['uniqueID']
#onbio_demo[['uniqueID', 'demo_illness','demo_illness_details','demo_meds','demo_meds_details']][55:105]
onbio_prior_diagnosis['prior_diagnosis'] = ['no']*len(onbio_prior_diagnosis)

# -------------------------------------
# handel with prior diagnosis manually
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '104', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '115', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '116', 'prior_diagnosis'] = 'other'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '121', 'prior_diagnosis'] = 'PTSD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '129', 'prior_diagnosis'] = 'other'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '132', 'prior_diagnosis'] = 'PTSD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '142', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '170', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '180', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '182', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '183', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '184', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '187', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '155', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '163', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '164', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '196', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '502', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '503', 'prior_diagnosis'] = 'PTSD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '504', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '505', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '509', 'prior_diagnosis'] = 'other'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '519', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '522', 'prior_diagnosis'] = 'ADHD'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '530', 'prior_diagnosis'] = 'DASS'
onbio_prior_diagnosis.loc[onbio_prior_diagnosis['uniqueID'] == '533', 'prior_diagnosis'] = 'DASS'


In [31]:
eng_onbio_demo = onbio_demo.replace(demo_dict)
eng_onbio_war = onbio_war.replace(war_dict)
len(eng_onbio_demo), len(eng_onbio_war)

(149, 149)

In [33]:

dataset_demo_war = pd.DataFrame()
dataset_demo_war['uniqueID'] = eng_onbio_demo['uniqueID']
eng_onbio_demo['demo_numChildren'] = pd.to_numeric(eng_onbio_demo['demo_numChildren'], errors='coerce')

# copy the demo data
dataset_demo_war = pd.merge(dataset_demo_war, onbio_prior_diagnosis, on='uniqueID')

dataset_demo_war = pd.merge(dataset_demo_war,eng_onbio_demo[['uniqueID','demo_age','demo_geneder','demo_eduction','demo_armyServed',
                                                             'demo_numChildren']], on='uniqueID')

dataset_demo_war = dataset_demo_war.rename(columns={'demo_age': 'age', 'demo_geneder': 'gender', 'demo_eduction':'education',
                                                    'demo_armyServed':'army_serve','demo_numChildren':'has_kids'})

# create the has kids a categorical assessment instad of numerical
dataset_demo_war['has_kids'] = np.where(dataset_demo_war['has_kids'] > 0, 'yes', dataset_demo_war['has_kids'])
dataset_demo_war['has_kids'] = np.where(dataset_demo_war['has_kids'] == 0, 'no', dataset_demo_war['has_kids'])

# copy the war data
dataset_demo_war = pd.merge(dataset_demo_war,eng_onbio_war[['uniqueID','war_alarmFreq_start','war_servForce','war_familiarFallens',
                                                             'war_jobRelate2war','war_jobSimilarities','war_newsConsumption','war_addressNow',
                                                             'war_familiarkidnap','war_worry2someone']], on='uniqueID')

dataset_demo_war = dataset_demo_war.rename(columns={'war_alarmFreq_start': 'sirens', 'war_servForce': 'reserve_duty', 
                                                    'war_familiarFallens':'knows_victims','war_jobRelate2war':'work_relate2war', 
                                                    'war_jobSimilarities':'work_changes','war_newsConsumption':'news_consumption',
                                                    'war_addressNow':'live_in_your_place','war_familiarkidnap':'knows_hostages',
                                                    'war_worry2someone':'worry_to_someone'})

# I created also war only dataset in case needed
dataset_war = pd.DataFrame()
dataset_war['uniqueID'] = eng_onbio_demo['uniqueID']
dataset_war = pd.merge(dataset_war,dataset_demo_war[['uniqueID','sirens','reserve_duty','knows_victims','work_relate2war','work_changes',
                                                     'news_consumption','live_in_your_place','knows_hostages','worry_to_someone']],
                        on='uniqueID')



In [35]:
dataset_demo_war


,uniqueID,prior_diagnosis,age,gender,education,army_serve,has_kids,sirens,reserve_duty,knows_victims,work_relate2war,work_changes,news_consumption,live_in_your_place,knows_hostages,worry_to_someone
0,101,no,25,female,during academy,yes,0.0,Few,NaN,No,No,Extremely,Two hours max,Yes,No,Yes
1,102,no,27,female,academic,yes,0.0,Few,NaN,No,No,Extremely,Two hours max,Yes,No,Yes
2,103,no,30,male,academic,yes,0.0,Between 2-3 times a week,NaN,No,No,A lot,Between 2 to 3 hours,Yes,No,Yes
3,104,DASS,28,male,academic,yes,0.0,Not at all,NaN,No,No,A lot,Between 2 to 3 hours,Yes,No,No
4,105,no,27,female,academic,yes,0.0,Few,NaN,No,No,A lot,Two hours max,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,553,no,24,male,during academy,yes,0.0,Few,"Yes, I finished my reserve",Yes,No,Slightly,Two hours max,Yes,No,Yes
145,554,no,46,male,academic,yes,0.0,Between 2-3 times a week,No,No,No,A lot,Two hours max,Yes,No,Yes
146,555,no,23,male,during academy,yes,0.0,Every day,"Yes, I finished my reserve",Yes,No,Not at all,Not at all,Yes,No,Yes
147,556,no,25,male,during academy,yes,0.0,Between 2-3 times a week,"Yes, I still serve",Yes,No,Extremely,Not at all,"No, I live at a house that not main",No,Yes


## Subjective Measures

### DASS & PCL & ASRS & COPE

In [39]:
# -------------------------------------------------
# ------------------ FUNCTIONS --------------------
# -------------------------------------------------

'''
 This function calculate the score for the DASS questionnaire 
 by sum all the relevant questions for each measure
 (1)Anxiety (2)Depression (3)Stress
 Input: the dataframe of the dass 
 Output: a new dataframe includs 3 columns one for each measure
'''
def calc_dass_scores(original_df):
    
    # Identify the groups based on column names
    anxiety_cols = [col for col in original_df.columns if col.startswith('dass_a')]
    depression_cols = [col for col in original_df.columns if col.startswith('dass_d')]
    stress_cols = [col for col in original_df.columns if col.startswith('dass_s')]

    # Calculate the sum for each group
    score_df = pd.DataFrame()
    score_df['uniqueID'] = original_df['uniqueID']
    score_df['anxiety'] = original_df[anxiety_cols].sum(axis=1)
    score_df['depression'] = original_df[depression_cols].sum(axis=1)
    score_df['stress'] = original_df[stress_cols].sum(axis=1)
    
    return score_df


'''
 This function calculate the score for the PCL questionnaire 
 by sum all the questions
 Input: the dataframe of the pcl 
 Output: a new dataframe includs a column of the score
'''
def calc_pcl_score(original_df):
    pcl_cols = [col for col in original_df.columns if col.startswith('pcl')]
    score_df = pd.DataFrame()
    score_df['uniqueID'] = original_df['uniqueID']
    score_df['pcl'] = original_df[pcl_cols].sum(axis=1)

    return score_df


'''
 This function calculate the score for the COPE questionnaire 
 by average the relevant questions for each cope mechanism
 (1)Problem focuser (2)Emotional focuser (3)Avoidance
 Input: the dataframe of the cope
 Output: a new dataframe includs 3 columns one for each measure
'''
def calc_cope_scores(original_df):
    
    # Identify the groups based on column names
    problems_cols = [col for col in original_df.columns if col.startswith('cope_p')]
    emotional_cols = [col for col in original_df.columns if col.startswith('cope_e')]
    avoidance_cols = [col for col in original_df.columns if col.startswith('cope_a')]

    # Calculate the sum for each group
    score_df = pd.DataFrame()
    score_df['uniqueID'] = original_df['uniqueID']
    score_df['problems'] = round((original_df[problems_cols].sum(axis=1))/(len(problems_cols)),2)
    score_df['emotional'] = round((original_df[emotional_cols].sum(axis=1))/(len(emotional_cols)),2)
    score_df['avoidance'] = round((original_df[avoidance_cols].sum(axis=1))/(len(avoidance_cols)),2)
    
    return score_df


'''
 This function calculate the score for the ASRS questionnaire 
 by sum all the questions
 Input: the dataframe of the asrs 
 Output: a new dataframe includs a column of the score
'''
def calc_asrs_score(original_df):
    asrs_cols = [col for col in original_df.columns if col.startswith('asrs')]
    score_df = pd.DataFrame()
    score_df['uniqueID'] = original_df['uniqueID']
    score_df['asrs'] = original_df[asrs_cols].sum(axis=1)

    return score_df



In [41]:
# create the second dataset table - dataset_subjective
dataset_subjective = pd.DataFrame()
dataset_subjective['uniqueID'] = onbio_dass['uniqueID']
numeric_onbio_dass = onbio_dass.replace(dass_dict)
numeric_onbio_pcl = onbio_pcl.replace(pcl_dict)
numeric_onbio_cope = onbio_cope.replace(cope_dict)
numeric_onbio_asrs = onbio_asrs.replace(asrs_dict)



# insert all the scores to the dataset
dass_scores = calc_dass_scores(numeric_onbio_dass)
dataset_subjective = pd.merge(dataset_subjective, dass_scores, on='uniqueID')

pcl_score = calc_pcl_score(numeric_onbio_pcl)
dataset_subjective = pd.merge(dataset_subjective, pcl_score, on='uniqueID')

asrs_score = calc_asrs_score(numeric_onbio_asrs)
dataset_subjective = pd.merge(dataset_subjective, asrs_score, on='uniqueID')

cope_scores = calc_cope_scores(numeric_onbio_cope)
dataset_subjective = pd.merge(dataset_subjective, cope_scores, on='uniqueID')

In [43]:
list(dataset_subjective['uniqueID'])

# need to remove 517, 525, 512
dataset_subjective_new = dataset_subjective.drop(dataset_subjective[dataset_subjective['uniqueID'] == '517'].index)
dataset_subjective_new = dataset_subjective_new.drop(dataset_subjective_new[dataset_subjective_new['uniqueID'] == '512'].index)
dataset_subjective_new = dataset_subjective_new.drop(dataset_subjective_new[dataset_subjective_new['uniqueID'] == '525'].index)
dataset_subjective_new = dataset_subjective_new.drop(dataset_subjective_new[dataset_subjective_new['uniqueID'] == '505'].index)

dataset_war_new = dataset_war.drop(dataset_war[dataset_war['uniqueID'] == '517'].index)
dataset_war_new = dataset_war_new.drop(dataset_war_new[dataset_war_new['uniqueID'] == '512'].index)
dataset_war_new = dataset_war_new.drop(dataset_war_new[dataset_war_new['uniqueID'] == '525'].index)
dataset_war_new = dataset_war_new.drop(dataset_war_new[dataset_war_new['uniqueID'] == '505'].index)

# dua 31 May we have until participant-541
print(len(list(dataset_subjective_new['uniqueID'])))
print(len(list(dataset_war_new['uniqueID'])))


145
145


# Read Flanker data

In [46]:
# reading the task csv for online and bios
'''
This function loop over all the csv files in a source folder
and copied them to the destionation folder
Input: (1)source path string (2)destionattion path string
Output:
'''
def copy_task_data(source_path, destionation_path):
    current_folder_path = source_path
    destination_folder = destionation_path
    csv_files_list = os.listdir(current_folder_path)
    for csv_file in csv_files_list:
        # get the uniqueID name from the original file name (3 letters number)
        new_file_name = csv_file[:3]
        shutil.copyfile(f'{current_folder_path}/{csv_file}', f'{destination_folder}/{new_file_name}.csv')
        


In [48]:
# save this data on my computer, if want to save in lab need to be change the folder name
# online_and_bio_task_data = '/Users/prague/Documents/NeuroScience-Master/Elana_Lab/War_Research/code/analysis/online_and_bio_task_data'
online_and_bio_task_data = '/Users/pagueoshri/Documents/NeuroScience-Master/Elana_Lab/War_Research/code/analysis/online_and_bio_task_data'
if not os.path.exists(online_and_bio_task_data):
    os.makedirs(online_and_bio_task_data)


source_path_online = '/Users/pagueoshri/Documents/NeuroScience-Master/Elana_Lab/War_Research/code/analysis/task_raw_data/online_only'
source_path_bio = '/Users/pagueoshri/Documents/NeuroScience-Master/Elana_Lab/War_Research/code/analysis/task_raw_data/bio'

# copy the data from online and bio to the same destionation folder
copy_task_data(source_path_online, online_and_bio_task_data)
copy_task_data(source_path_bio, online_and_bio_task_data)

# count the number of csv files in the new folder
onbio_csv_files_list = os.listdir(online_and_bio_task_data)

# dua 31 May we have until participant-538
onbio_csv_files_list = sorted(onbio_csv_files_list)
onbio_csv_files_list.remove('.DS_Store')
onbio_csv_files_list.remove('505.csv')
len(onbio_csv_files_list)


144

### Global variables

In [51]:
Num_Trials = 256
print(type(onbio_csv_files_list[1]))
t = pd.read_csv(f'online_and_bio_task_data/{onbio_csv_files_list[1]}')
t['trial_resp.keys'][16:]

<class 'str'>


16     right
17     right
18     right
19     right
20      left
       ...  
268    right
269    right
270    right
271     left
272      NaN
Name: trial_resp.keys, Length: 257, dtype: object

In [53]:
'''
This function getting the same group of random trials 
for the no-vol condition, in order to get the same 
group size of analysis
Input: csv file
Output: list of indeces from the no-vol trials
'''
def get_random_trials_of_noVol(csv_file):
    tone_group = csv_file[(csv_file['vol'] == 1)]
    no_tone_group = csv_file[(csv_file['vol'] == 0)]
    
    # rand trials as the number of 25% trials with tone
    seed(1) # choose the same trials accross all the random
    random_trials = randint(1, len(no_tone_group), len(tone_group))
    
    return random_trials



'''
This function reading the relevant information from each csv
'''
def fill_task_table_variables(csv_file, unique_id):
#     print(csv_file['trial_resp.keys'])
    destination_df = pd.DataFrame()
    
    num_misses = csv_file['trial_resp.keys'].isna().sum()
#     print(num_misses)
    mean_rt = np.mean(csv_file['trial_resp.rt'])
    acc = (np.sum(csv_file['trial_resp.corr']) / (Num_Trials)) * 100 # accuracy percentage
    
    con_rows = csv_file.loc[(csv_file['conditions'].eq('con_right') | csv_file['conditions'].eq('con_left'))]
    con_rt = np.mean(con_rows['trial_resp.rt'])
    con_acc = (np.sum(con_rows['trial_resp.corr']))/(len(con_rows)) * 100
    
    incon_rows = csv_file.loc[csv_file['conditions'].eq('incon_right') | csv_file['conditions'].eq('incon_left')]
    incon_rt = np.mean(incon_rows['trial_resp.rt'])
    incon_acc = (np.sum(incon_rows['trial_resp.corr']))/(len(incon_rows)) * 100
    
    vol_rows = csv_file.loc[csv_file['vol'] == 1]
    vol_rt = np.mean(vol_rows['trial_resp.rt'])
    vol_acc = (np.sum(vol_rows['trial_resp.corr']))/(len(vol_rows)) * 100
    
    random_trials = get_random_trials_of_noVol(csv_file)
    novol_rows = csv_file[(csv_file['vol'] == 0)].iloc[random_trials]
    novol_rt = np.mean(novol_rows['trial_resp.rt'])
    novol_acc = (np.sum(novol_rows['trial_resp.corr']))/(len(novol_rows)) * 100
    
    phasic_alertness = novol_rt - vol_rt
    executive_control = con_rt - incon_rt

    # find the union group of the novol/vol with con/incon 
    # in order to get comparisions of phasic alertness and executive control
    vol_con_rows = pd.merge(con_rows, vol_rows, how='inner')
    novol_con_rows = pd.merge(con_rows, novol_rows, how='inner')
    vol_incon_rows = pd.merge(incon_rows, vol_rows, how='inner')
    novol_incon_rows = pd.merge(incon_rows, novol_rows, how='inner')
    
    vol_con_rt = np.mean(vol_con_rows['trial_resp.rt'])
    novol_con_rt = np.mean(novol_con_rows['trial_resp.rt'])
    vol_incon_rt = np.mean(vol_incon_rows['trial_resp.rt'])
    novol_incon_rt = np.mean(novol_incon_rows['trial_resp.rt'])

    phasic_alertness_con = novol_con_rt - vol_con_rt
    phasic_alertness_incon = novol_incon_rt - vol_incon_rt
    executive_control_vol = vol_con_rt - vol_incon_rt
    executive_control_novol = novol_con_rt - novol_incon_rt

    
    task_dict = {'uniqueID':unique_id, 'misses':num_misses, 'mean_RT':mean_rt, 'accuracy': acc,
                 'con_RT':con_rt, 'incon_RT':incon_rt, 'vol_RT':vol_rt, 'novol_RT':novol_rt,
                 'con_Acc':con_acc, 'incon_Acc':incon_acc, 'vol_Acc':vol_acc, 'novol_Acc':novol_acc,
                 'phasic_alertness': phasic_alertness, 'executive_control': executive_control,
                 'phasic_alertness_con': phasic_alertness_con, 'phasic_alertness_incon': phasic_alertness_incon,
                 'executive_control_vol': executive_control_vol, 'executive_control_novol': executive_control_novol}
    
    return task_dict
    


In [57]:
# create a new table for the task data
columns=['uniqueID','misses','mean_RT','accuracy','con_RT','incon_RT','vol_RT','novol_RT','con_Acc',
         'incon_Acc','vol_Acc','novol_Acc','phasic_alertness', 'executive_control', 'phasic_alertness_con', 
         'phasic_alertness_incon','executive_control_vol', 'executive_control_novol']
index=list(dataset_subjective_new['uniqueID'])
dataset_objective_task = pd.DataFrame(columns=columns,index=index)
dataset_objective_task['uniqueID'] = index
dataset_objective_task = dataset_objective_task.drop(dataset_objective_task[dataset_objective_task['uniqueID'] == '505'].index)


# loop over all the csv files and fill in the table
for csv_file in onbio_csv_files_list:
    csv_unique_id = csv_file[:3]
    curr_csv_file = pd.read_csv(f'online_and_bio_task_data/{csv_file}')
    curr_dict = fill_task_table_variables(curr_csv_file[16:],csv_unique_id)
    dataset_objective_task.loc[curr_dict['uniqueID']] = pd.Series(curr_dict)

print(len(dataset_objective_task))
print(len(dataset_objective_task[dataset_objective_task['misses'] >= 20]))
# Remove outlier where misses more than 20 trials
# dataset_objective_task = dataset_objective_task.drop(dataset_objective_task[dataset_objective_task['misses'] >= 10].index)
dataset_objective_task = dataset_objective_task.drop(dataset_objective_task[dataset_objective_task['misses'] >= 20].index)
# dataset_objective_task = dataset_objective_task.drop(dataset_objective_task[dataset_objective_task['misses'] >= 35].index)
print(len(dataset_objective_task))

145
6
139


In [59]:
dataset_objective_task

# Need to remove participants that didn't use headphones in the task:
# uniqueIDs: 101, 102, 185
dataset_objective_task_new = dataset_objective_task.drop(dataset_objective_task[dataset_objective_task['uniqueID'] == '101'].index)
dataset_objective_task_new = dataset_objective_task_new.drop(dataset_objective_task_new[dataset_objective_task_new['uniqueID'] == '102'].index)
dataset_objective_task_new = dataset_objective_task_new.drop(dataset_objective_task_new[dataset_objective_task_new['uniqueID'] == '185'].index)

len(dataset_objective_task_new['uniqueID'])

136

## Save 3 DataFrames as csv files

In [65]:
dataset_demo_war.to_csv('demo_war_table.csv')
dataset_war_new.to_csv('war_table.csv')
dataset_subjective_new.to_csv('subjective_table.csv')
dataset_objective_task_new.to_csv('objective_task_table.csv')